In [7]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import copy
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.utils.data as data_utils

from models.embedding_net import EmbeddingNet

In [2]:
resource_path = os.path.join(".", 'data', 'ml-25m')
movies_csv = os.path.join(resource_path, 'movies.csv')
links_csv = os.path.join(resource_path, 'links.csv')

In [5]:
movies = pd.read_csv(movies_csv)
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
movies['genres'] = movies.genres.apply(lambda genres: genres.split('|'))
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),[Comedy]


In [11]:
links = pd.read_csv(links_csv)
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [12]:
movies = movies.merge(links)
del links

movies.head()

,movieId,title,genres,imdbId,tmdbId
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",114709,862.0
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]",113497,8844.0
2,3,Grumpier Old Men (1995),"[Comedy, Romance]",113228,15602.0
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",114885,31357.0
4,5,Father of the Bride Part II (1995),[Comedy],113041,11862.0


In [8]:
model_path = os.path.join(".", 'models', 'binaries')
models = [model for model in os.listdir(model_path) if model.endswith(".pth")]
models.sort()
models

final_model = models[-1]
final_model_path = os.path.join(model_path, final_model)
final_model_path

'.\\models\\binaries\\Embedding_weights_Final.pth'

In [16]:
embedding_state = torch.load(final_model_path)
embedding_state

state_dict = embedding_state['state_dict']
embedding_net = EmbeddingNet(embedding_state['n_users'], embedding_state['n_movies'], embedding_state['n_factors'])
embedding_net.load_state_dict(state_dict, strict=True)
embedding_net.eval()

EmbeddingNet(
  (user_factors): Embedding(160543, 10)
  (user_bias): Embedding(160543, 1)
  (movie_factors): Embedding(200, 10)
  (movie_bias): Embedding(200, 1)
)

In [19]:
def get_movie_id(label):
    id = embedding_state['movieLabel_to_Id']([label])
    return id.item(0)

label = 24
embedding_net.movie_factors.weight[label]
movies[movies['movieId']==get_movie_id(label)]['title'].item()

'Star Wars: Episode IV - A New Hope (1977)'